In [10]:
# Importando librerias 
import numpy as np
import math
from UQpy.distributions import Uniform, JointIndependent
from UQpy.surrogates import *

In [2]:
# function to be approximated
def ishigami(xx):
    """Ishigami function"""
    a = 7
    b = 0.1
    term1 = np.sin(xx[0])
    term2 = a * np.sin(xx[1])**2
    term3 = b * xx[2]**4 * np.sin(xx[0])
    return term1 + term2 + term3

In [3]:
# input distributions
dist1 = Uniform(loc=-np.pi, scale=2*np.pi)
dist2 = Uniform(loc=-np.pi, scale=2*np.pi)
dist3 = Uniform(loc=-np.pi, scale=2*np.pi)
marg = [dist1, dist2, dist3]
joint = JointIndependent(marginals=marg)

In [4]:
# maximum polynomial degree
P = 6

# construct total-degree polynomial basis
polynomial_basis = TotalDegreeBasis(joint, P)

# check the size of the basis
print('Size of PCE basis:', polynomial_basis.polynomials_number)

Size of PCE basis: 84


In [5]:
# create training data
sample_size = int(polynomial_basis.polynomials_number*5)
print('Size of experimental design:', sample_size)

# realizations of random inputs
xx_train = joint.rvs(sample_size)
# corresponding model outputs
yy_train = np.array([ishigami(x) for x in xx_train])

Size of experimental design: 420


In [6]:
# fit model
least_squares = LeastSquareRegression()
pce = PolynomialChaosExpansion(polynomial_basis=polynomial_basis, regression_method=least_squares)

pce.fit(xx_train, yy_train)

In [7]:
mean_est = pce.get_moments()[0]
var_est = pce.get_moments()[1]
print('PCE mean estimate:', mean_est)
print('PCE variance estimate:', var_est)

PCE mean estimate: 3.4815562290824356
PCE variance estimate: 13.727464306463007


In [8]:
from UQpy.sensitivity import *
pce_sensitivity = PceSensitivity(pce)
pce_sensitivity.run()
sobol_first = pce_sensitivity.first_order_indices
sobol_total = pce_sensitivity.total_order_indices
print('First-order Sobol indices:')
print(sobol_first)
print('Total-order Sobol indices:')
print(sobol_total)

First-order Sobol indices:
[[2.95688216e-01]
 [4.41936493e-01]
 [4.12538794e-04]]
Total-order Sobol indices:
[[0.55608976]
 [0.44742436]
 [0.25985598]]


In [9]:
# validation data sets
np.random.seed(999)  # fix random seed for reproducibility
n_samples_val = 100000
xx_val = joint.rvs(n_samples_val)
yy_val = np.array([ishigami(x) for x in xx_val])

mae = []  # to hold MAE for increasing polynomial degree
for degree in range(16):
    # define PCE
    polynomial_basis = TotalDegreeBasis(joint, degree)
    least_squares = LeastSquareRegression()
    pce_metamodel = PolynomialChaosExpansion(polynomial_basis=polynomial_basis, regression_method=least_squares)

    # create training data
    np.random.seed(1)  # fix random seed for reproducibility
    sample_size = int(pce_metamodel.polynomials_number * 5)
    xx_train = joint.rvs(sample_size)
    yy_train = np.array([ishigami(x) for x in xx_train])

    # fit PCE coefficients
    pce_metamodel.fit(xx_train, yy_train)

    # compute mean absolute validation error
    yy_val_pce = pce_metamodel.predict(xx_val).flatten()
    errors = np.abs(yy_val.flatten() - yy_val_pce)
    mae.append(np.linalg.norm(errors, 1) / n_samples_val)

    print('Polynomial degree:', degree)
    print('Mean absolute error:', mae[-1])
    print(' ')

Polynomial degree: 0
Mean absolute error: 3.5092014513593974
 
Polynomial degree: 1
Mean absolute error: 2.9190940130421437
 
Polynomial degree: 2
Mean absolute error: 2.881426057510191
 
Polynomial degree: 3
Mean absolute error: 2.490517639729513
 
Polynomial degree: 4
Mean absolute error: 1.6837629839128996
 
Polynomial degree: 5
Mean absolute error: 1.4288047926039877
 
Polynomial degree: 6
Mean absolute error: 0.47225689340879823
 
Polynomial degree: 7
Mean absolute error: 0.3275845356142154
 
Polynomial degree: 8
Mean absolute error: 0.06852103811265503
 
Polynomial degree: 9
Mean absolute error: 0.044242183981870506
 
Polynomial degree: 10
Mean absolute error: 0.0049732041095262
 
Polynomial degree: 11
Mean absolute error: 0.0038931663654602523
 
Polynomial degree: 12
Mean absolute error: 0.00025446069874188036
 
Polynomial degree: 13
Mean absolute error: 0.0002543042852671317
 
Polynomial degree: 14
Mean absolute error: 1.1189147899934252e-05
 
Polynomial degree: 15
Mean absolut